In [9]:
import os
import tarfile

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import scipy.io

from PIL import Image

In [ ]:
# with tarfile.open(r'<path>', 'r:gz') as tar:
#     tar.list()
#     tar.extractall(path='./data')

## 1 Load & Preprocess Data


In [10]:
class FlowerDataset(Dataset):
    def __init__(self, image_dir, label_file, transform=None):
        self.image_dir = image_dir
        self.transform = transform
        
        # Load image labels
        mat = scipy.io.loadmat(label_file)
        self.labels = mat['labels'].flatten() - 1 # converts 1-based idx in matlab to 0-based idx in python
        
        # Load image file names
        self.image_files = [f for f in os.listdir(image_dir) if f.endswith('.jpg')]
        self.image_files.sort()
    
    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_file[idx]
        img_path = os.path.join(self.image_dir, img_name)
        image = Image.open(img_path).convert('RGB')
        
        label = self.labels[idx]
        return image, torch.tensor(label, dtype=torch.long)
        

In [18]:
image_size = 64
batch_size = 64

transform = transforms.Compose(
    [
        transforms.Resize((image_size, image_size)), # resize image to (image_size x image_size)
        transforms.ToTensor(), # convert arrays to tensors
        transforms.Normalize([.5, .5, .5], [.5, .5, .5]), # normalize pixels (pixel - mean) / STD -> mean & SD for R,G,B is 0.5
    ]
)

img_dir = "./data/jpg"
label_file = "imagelabels.mat"

dataset = FlowerDataset(img_dir, label_file, transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

## 2 Define Conditional GAN models


In [ ]:
class ConditionalGenerator(nn.Module):
    def __init__(self, latent_dim, label_dim, num_classes):
        super(ConditionalGenerator, self).__init__()
        self.label_embedding = nn.Embedding(num_classes, label_dim) # create embedding layer to map label indices (102 different flowers)
        self.model = nn.Sequential( # focus on linear layers only first
            nn.Linear(latent_dim + label_dim, 256), # input to CondGenerator is noise & label, 256 output
            nn.ReLU(), # introduce non-linearity
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, image_size * image_size), # output is 64 x 64 vector
            nn.Tanh() # good practice to add at end to further normalize inputs
        )
    
    def forward(self, noise, labels):
        label_emb = self.label_embedding(labels) # map label indices (102 different flowers) by passing to embedding layer
        generator_input = torch.cat((noise, label_emb), -1)  # concat tensors, -1 means combine by col to prep input
        img = self.model(generator_input)
        img = img.view(img.size(0), 3, image_size, image_size) # reshape to (batchsize, RGB, image_size, image_size) (64, 3, 64, 64)
        return img
        